In [3]:
import pandas as pd
import numpy as np
url = 'http://statweb.stanford.edu/~tibs/ElemStatLearn/datasets/prostate.data'
df = pd.read_csv(url, sep='\t', header=0)
df = df.drop('Unnamed: 0', axis=1) #elimina columna redundante

istrain_str = df['train']
istrain = np.asarray([True if s == 'T' else False for s in istrain_str])
istest = np.logical_not(istrain)

df = df.drop('train', axis=1) #elimina columna que no entrega mas informacion

In [4]:
df.shape
df.info()
df.describe()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 97 entries, 0 to 96
Data columns (total 9 columns):
lcavol     97 non-null float64
lweight    97 non-null float64
age        97 non-null int64
lbph       97 non-null float64
svi        97 non-null int64
lcp        97 non-null float64
gleason    97 non-null int64
pgg45      97 non-null int64
lpsa       97 non-null float64
dtypes: float64(5), int64(4)
memory usage: 7.6 KB


,lcavol,lweight,age,lbph,svi,lcp,gleason,pgg45,lpsa
count,97.000000,97.000000,97.000000,97.000000,97.000000,97.000000,97.000000,97.000000,97.000000
mean,1.350010,3.628943,63.865979,0.100356,0.216495,-0.179366,6.752577,24.381443,2.478387
std,1.178625,0.428411,7.445117,1.450807,0.413995,1.398250,0.722134,28.204035,1.154329
min,-1.347074,2.374906,41.000000,-1.386294,0.000000,-1.386294,6.000000,0.000000,-0.430783
25%,0.512824,3.375880,60.000000,-1.386294,0.000000,-1.386294,6.000000,0.000000,1.731656
50%,1.446919,3.623007,65.000000,0.300105,0.000000,-0.798508,7.000000,15.000000,2.591516
75%,2.127041,3.876396,68.000000,1.558145,0.000000,1.178655,7.000000,40.000000,3.056357
max,3.821004,4.780383,79.000000,2.326302,1.000000,2.904165,9.000000,100.000000,5.582932


estandarizar datos

In [5]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
df_scaled = pd.DataFrame(scaler.fit_transform(df), columns=df.columns)
df_scaled['lpsa'] = df['lpsa']

Ajuste lineal sobre Xtrain e ytrain, guardando en linreg

In [6]:
import sklearn.linear_model as lm
X = df_scaled.ix[:,:-1] #se le elimina la columna del target
N = X.shape[0]
X.insert(X.shape[1], 'intercept', np.ones(N))
y = df_scaled['lpsa'] #columna target
Xtrain = X[istrain]
ytrain = y[istrain]
Xtest = X[np.logical_not(istrain)]
ytest = y[np.logical_not(istrain)]
linreg = lm.LinearRegression(fit_intercept = False)
linreg.fit(Xtrain, ytrain)
print type(Xtrain)

<class 'pandas.core.frame.DataFrame'>


tabla de pesos y Z-score

In [10]:
yhat_model = linreg.predict(Xtrain)
Xm = Xtrain.as_matrix()
mse_model = np.mean( np.power( (yhat_model - ytrain) , 2) ) #error del modelo (train)
var_est = mse_model * np.diag(np.linalg.pinv(np.dot(Xm.T,Xm)))
std_err = np.sqrt(var_est)

names_regressors = ["Lcavol", "Lweight", "Age", "Lbph", "Svi", "Lcp", "Gleason", "Pgg45", "Intercept"]
table = [names_regressors, linreg.coef_, std_err, linreg.coef_/std_err]
table = zip(*table) #traspuesta

from tabulate import tabulate
print tabulate(table, headers=["Atributo","Coeficiente", "Std. Err","Z-score"],  tablefmt="rst")

==========  =============  ==========  =========
Atributo      Coeficiente    Std. Err    Z-score
==========  =============  ==========  =========
Lcavol          0.676016    0.117209    5.76763
Lweight         0.261694    0.0885141   2.95652
Age            -0.140734    0.0938032  -1.50031
Lbph            0.209061    0.0946146   2.2096
Svi             0.303623    0.114405    2.65393
Lcp            -0.287002    0.143033   -2.00654
Gleason        -0.0211949   0.134442   -0.157651
Pgg45           0.265576    0.142186    1.86781
Intercept       2.46493     0.0831     29.6623
==========  =============  ==========  =========


cross-validation

In [11]:
from sklearn import cross_validation
def C_V(Xm,ym, K=10):
    k_fold = cross_validation.KFold(len(Xm),K)
    mse_cv = 0
    for k, (train, val) in enumerate(k_fold):
        linreg = lm.LinearRegression(fit_intercept = False)
        linreg.fit(Xm[train], ym[train])
        yhat_val = linreg.predict(Xm[val])
        mse_fold = np.mean(np.power(yhat_val - ym[val], 2))
        mse_cv += mse_fold
    mse_cv = mse_cv / K
    return mse_cv

yhat_test = linreg.predict(Xtest)
mse_test = np.mean(np.power(yhat_test - ytest, 2))

Xm = Xtrain.as_matrix()
ym = ytrain.as_matrix()

mse_cv = C_V(Xm,ym) # k = 10
mse_cv_d =  C_V(Xm,ym,5) # k = 5
table = [[mse_test,C_V(Xm,ym),C_V(Xm,ym,5)]]
print tabulate(table, headers= ["Test error","CV k=10","CV k=5"], tablefmt="rst")

ep = (mse_cv - mse_test)/mse_test
print "error porcentual de K =10 y K = 5"
print ep*100
ep = (mse_cv_d - mse_test)/mse_test
print ep*100

============  =========  ========
  Test error    CV k=10    CV k=5
============  =========  ========
    0.521274   0.757237  0.956515
============  =========  ========
error porcentual de K =10 y K = 5
45.266686035
83.4955554104


error de entrenamiento y QQ-plot

In [22]:
yhat_train = linreg.predict(Xtrain) #predicto por modelo
residuo =(yhat_train - ytrain)

# QQplot
import pylab 
import scipy.stats as stats
  
stats.probplot(residuo,dist="norm", plot=pylab)
pylab.title("Residuos")
pylab.show()

----------------------------------------------------------------------------------------------------------------------

Seleccionar atributos/caracteristicas  #para ver como se comporta el modelo con menos caracteristicas

In [17]:
def fss(x, y, names_x, k = 10000):
    p = x.shape[1]-1
    k = min(p, k)
    names_x = np.array(names_x)
    remaining = range(0, p)
    selected = [p]
    current_score = 0.0
    best_new_score = 0.0
    while remaining and len(selected)<=k :
        score_candidates = []
        for candidate in remaining:
            model = lm.LinearRegression(fit_intercept=False)
            indexes = selected + [candidate]
            x_train = x[:,indexes]
            
            #calcular error mediante cross validation
            mse_candidate = C_V(x_train,y, K=10)
            score_candidates.append((mse_candidate, candidate))
        score_candidates.sort()
        score_candidates[:] = score_candidates[::-1]
        best_new_score, best_candidate = score_candidates.pop() #el de menor error es el mejor candidato
        remaining.remove(best_candidate)
        selected.append(best_candidate)
        print "selected = %s ..."%names_x[best_candidate]
        print "totalvars=%d, mse = %f"%(len(indexes),best_new_score)
    return selected
names_regressors = ["Lcavol", "Lweight", "Age", "Lbph", "Svi", "Lcp", "Gleason", "Pgg45"]
sequence = fss(Xm,ym,names_regressors)
print sequence

selected = Lcavol ...
totalvars=2, mse = 0.876172
selected = Lweight ...
totalvars=3, mse = 0.752606
selected = Lbph ...
totalvars=4, mse = 0.748883
selected = Svi ...
totalvars=5, mse = 0.746635
selected = Pgg45 ...
totalvars=6, mse = 0.748007
selected = Lcp ...
totalvars=7, mse = 0.734094
selected = Age ...
totalvars=8, mse = 0.726706
selected = Gleason ...
totalvars=9, mse = 0.757237
[8, 0, 1, 3, 4, 7, 5, 2, 6]


Graficar FSS

In [18]:
mse_trains = {}
mse_tests = {}
aux = []
model = lm.LinearRegression(fit_intercept=False)
for index in sequence:
    aux = aux + [index]     
    #calcular error de training set
    x_train = Xtrain.as_matrix()[:,aux]
    predictions_train = model.fit(x_train, ym).predict(x_train)
    residuals_train = predictions_train - ym
    mse_train = np.mean(np.power(residuals_train, 2))
    mse_trains[len(aux) -1] = mse_train
    
    #calcular error de test set
    x_test = Xtest.as_matrix()[:,aux]
    predictions_test = model.fit(x_train, ym).predict(x_test)
    residuals_test= predictions_test - ytest
    mse_test = np.mean(np.power(residuals_test, 2))
    mse_tests[len(aux)-1] = mse_test

mse_trains.pop(0) #sin contar 0 variables
mse_tests.pop(0) #sin contar 0 variables
import matplotlib.pylab as plt
ax = plt.gca()
ax.plot(range(1,9),mse_trains.values(), label='train error')
ax.plot(range(1,9),mse_tests.values() , label='test error')
plt.legend(loc=2)

plt.xlabel('number variable')
plt.ylabel('mean square error')
plt.title('Error on FSS')
plt.axis('tight')
plt.show()

In [19]:
def bss(x, y, names_x, k = 10000):
    p = x.shape[1]-1 #numero de caracteristicas
    k = min(p, k)
    names_x = np.array(names_x)
    removing = [] #orden en que se eliminan
    selected =  range(0, p) #cambio
    current_score = 0.0
    best_new_score = 0.0
    while len(selected)>0 : #cambio
        score_candidates = [] #candidatos a ser eliminados
        for candidate in selected:
            model = lm.LinearRegression(fit_intercept=False)
            
            indexes = [p] + selected  # p intercepto
            indexes.remove(candidate) #elimina el posible candidato a ser eliminado indexes = selected - candidate
            
            x_train = x[:,indexes] #datos a probar
            
            #calcular error mediante cross validation
            mse_candidate = C_V(x_train,y, K=10)
            score_candidates.append((mse_candidate, candidate))
            #print "viendo que tal es eliminar: "+ str(candidate)+ " entrega un error de: "+str(mse_candidate)
        score_candidates.sort()
        score_candidates[:] = score_candidates[::-1]
        #se elige el modelo con el menor error localmente, elimmando el candidato que fue elimiando de ese modelo
        best_new_score, best_candidate = score_candidates.pop() 
        selected.remove(best_candidate)  #cambio
        removing.append(best_candidate) #cambio
        print "selected to delete = %s ..."%names_x[best_candidate]
        print "totalvars=%d, mse = %f"%(len(indexes),best_new_score)
    return removing
names_regressors = ["Lcavol", "Lweight", "Age", "Lbph", "Svi", "Lcp", "Gleason", "Pgg45"]
sequence_bss = bss(Xm,ym,names_regressors) #secuencia de eliminacion
print sequence_bss

selected to delete = Gleason ...
totalvars=8, mse = 0.726706
selected to delete = Age ...
totalvars=7, mse = 0.734094
selected to delete = Lcp ...
totalvars=6, mse = 0.748007
selected to delete = Pgg45 ...
totalvars=5, mse = 0.746635
selected to delete = Svi ...
totalvars=4, mse = 0.748883
selected to delete = Lbph ...
totalvars=3, mse = 0.752606
selected to delete = Lweight ...
totalvars=2, mse = 0.876172
selected to delete = Lcavol ...
totalvars=1, mse = 1.795596
[6, 2, 5, 7, 4, 3, 1, 0]


Grafico BSS

In [21]:
mse_trains = {}
mse_tests = {}
model = lm.LinearRegression(fit_intercept=False)
aux = [Xm.shape[1]-1] + sequence_bss
for index in sequence_bss:
    #calcular error de training set y test set
    x_train = Xtrain.as_matrix()[:,aux]
    yhat_train = model.fit(x_train, ym).predict(x_train)
    residuals_train = yhat_train - ym
    mse_train = np.mean(np.power(residuals_train, 2))
    mse_trains[len(aux) -1] = mse_train
        
    x_test = Xtest.as_matrix()[:,aux]
    yhat_test = model.fit(x_train, ym).predict(x_test)
    residuals_test= yhat_test - ytest
    mse_test = np.mean(np.power(residuals_test, 2))
    mse_tests[len(aux)-1] = mse_test
    
    aux.remove(index)
print mse_trains
ax = plt.gca()
ax.plot(range(1,9),mse_trains.values(), label='train error')
ax.plot(range(1,9),mse_tests.values() , label='test error')
plt.legend(loc=2)

plt.xlabel('number variable')
plt.ylabel('mean square error')
plt.title('Error on BSS')
plt.axis('tight')
plt.show()

{1: 0.66460571129035551, 2: 0.55360963630688553, 3: 0.53753980754784947, 4: 0.48977604102709787, 5: 0.47864846764674296, 6: 0.4558175840171248, 7: 0.43936269130473249, 8: 0.43919976805833433}
